In [253]:
import numpy as np 
import pandas as pd
import math
import matplotlib.pyplot as plt

In [254]:
df = pd.read_csv('/home/praneeth/Desktop/ACM/aiml/Models/framingham.csv')

In [255]:
X_train = df.iloc[:, :-1].values
y_train = df.iloc[:, -1].values

In [256]:
X_train = np.nan_to_num(X_train)

In [257]:
print("First five elements in X_train are:\n", X_train[:5])
print("Type of X_train:",type(X_train))

First five elements in X_train are:
 [[  1.    39.     4.     0.     0.     0.     0.     0.     0.   195.
  106.    70.    26.97  80.    77.  ]
 [  0.    46.     2.     0.     0.     0.     0.     0.     0.   250.
  121.    81.    28.73  95.    76.  ]
 [  1.    48.     1.     1.    20.     0.     0.     0.     0.   245.
  127.5   80.    25.34  75.    70.  ]
 [  0.    61.     3.     1.    30.     0.     0.     1.     0.   225.
  150.    95.    28.58  65.   103.  ]
 [  0.    46.     3.     1.    23.     0.     0.     0.     0.   285.
  130.    84.    23.1   85.    85.  ]]
Type of X_train: <class 'numpy.ndarray'>


In [258]:
print("First five elements in y_train are:\n", y_train[:5])
print("Type of y_train:",type(y_train))

First five elements in y_train are:
 [0 0 0 1 0]
Type of y_train: <class 'numpy.ndarray'>


In [259]:
print ('The shape of X_train is: ' + str(X_train.shape))
print ('The shape of y_train is: ' + str(y_train.shape))
print ('We have m = %d training examples' % (len(y_train)))

The shape of X_train is: (4238, 15)
The shape of y_train is: (4238,)
We have m = 4238 training examples


In [260]:
def zscore_normalize_features(X):
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    X_norm = (X - mu) / sigma
    return X_norm, mu, sigma

In [261]:
X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)
print("First five elements in X_norm are:\n", X_norm[:5])
print("Type of X_norm:",type(X_norm))

First five elements in X_norm are:
 [[ 1.1531919  -1.23495068  1.96608614 -0.98827076 -0.75132224 -0.17361158
  -0.07703255 -0.67110093 -0.16247659 -0.76087026 -1.19590711 -1.08262515
   0.29039572  0.34262779  0.07720798]
 [-0.86715836 -0.41825733  0.06655962 -0.98827076 -0.75132224 -0.17361158
  -0.07703255 -0.67110093 -0.16247659  0.31411609 -0.51518725 -0.15898843
   0.68854851  1.58433854  0.0467814 ]
 [ 1.1531919  -0.18491638 -0.88320363  1.01186844  0.92920959 -0.17361158
  -0.07703255 -0.67110093 -0.16247659  0.21639006 -0.22020864 -0.24295541
  -0.07834806 -0.0712758  -0.13577807]
 [-0.86715836  1.33179984  1.01632288  1.01186844  1.76947551 -0.17361158
  -0.07703255  1.49008884 -0.16247659 -0.17451407  0.80087114  1.0165492
   0.65461504 -0.89908297  0.86829905]
 [-0.86715836 -0.41825733  1.01632288  1.01186844  1.18128937 -0.17361158
  -0.07703255 -0.67110093 -0.16247659  0.99819832 -0.10675533  0.09291249
  -0.58508798  0.75653137  0.32062062]]
Type of X_norm: <class 'numpy

In [262]:
def sigmoid(z):
    g = 1/(1+np.exp(-z)) 
    return g

In [263]:
def compute_cost(X, y, w, b):
    m, n = X.shape
    cost = 0.0
    for i in range(m):
        z_i = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z_i)
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
    total_cost = cost / m
    return total_cost

In [264]:
m, n = X_norm.shape
initial_w = np.zeros(n)
initial_b = 0.
cost = compute_cost(X_norm, y_train, initial_w, initial_b)
print(f'Cost at initial w and b (zeros): {cost}')

Cost at initial w and b (zeros): 0.693147180559969


In [265]:
def compute_cost_reg(X, y, w, b, lambda_):
    m, n = X.shape
    cost_without_reg = compute_cost(X, y, w, b) 
    reg_cost = 0.
    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j]**2)
    reg_cost = (lambda_/(2*m)) * reg_cost
    total_cost = cost_without_reg + reg_cost
    return total_cost

In [266]:
m, n = X_norm.shape
initial_w = np.zeros(n)
initial_b = 0.
lambda_ = 1
cost = compute_cost_reg(X_norm, y_train, initial_w, initial_b, lambda_)
print("Regularized cost :", cost)

Regularized cost : 0.693147180559969


In [267]:
def compute_gradient(X, y, w, b):
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.
    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i],w) + b)
        err_i  = f_wb_i  - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m
    dj_db = dj_db/m
    return dj_db, dj_dw

In [268]:
dj_db, dj_dw = compute_gradient(X_norm, y_train, initial_w, initial_b)
print(f'dj_db at initial w and b (zeros):{dj_db}' )
print(f'dj_dw at initial w and b (zeros):{dj_dw.tolist()}' )

dj_db at initial w and b (zeros):0.34804152902312413
dj_dw at initial w and b (zeros):[-0.03174378400474678, -0.08086257849115808, 0.01834722411410884, -0.006984422765257179, -0.021129291464553182, -0.03102207930059881, -0.02218857350040424, -0.063755940583722, -0.034934743465037034, -0.023852214763711733, -0.07769383327302101, -0.0521595641669032, -0.014889891819714278, -0.0069245061692021475, -0.03530259622765956]


In [269]:
def compute_gradient_reg(X, y, w, b, lambda_): 
    m, n = X.shape
    dj_db, dj_dw = compute_gradient(X, y, w, b)
    for j in range(n):
        dj_dw[j] = dj_dw[j] + (lambda_/m) * w[j]       
    return dj_db, dj_dw

In [270]:
m, n = X_norm.shape
initial_w = np.zeros(n)
initial_b = 0.
lambda_ = 1
dj_db, dj_dw = compute_gradient_reg(X_norm, y_train, initial_w, initial_b, lambda_)
print(f"dj_db: {dj_db}", )
print(f"First few elements of regularized dj_dw:\n {dj_dw[:4].tolist()}", )

dj_db: 0.34804152902312413
First few elements of regularized dj_dw:
 [-0.03174378400474678, -0.08086257849115808, 0.01834722411410884, -0.006984422765257179]


In [271]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    m = len(X)
    J_history = []
    for i in range(num_iters):
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)   
        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
        if i<100000:
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            print(f"Iteration {i:4}: Cost {float(J_history[-1])}   ")
    return w_in, b_in, J_history

In [272]:
def predict(X, w, b): 
    m, n = X.shape
    p = np.zeros(m) 
    for i in range(m):   
        z_i = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z_i) 
        if f_wb_i >= 0.5:
            p[i] = 1
        else:
            p[i] = 0
    return p

In [273]:
def predict_single_example(X_i, w, b): 
    z = np.dot(X_i,w) + b
    f_wb = sigmoid(z) 
    if f_wb >= 0.5:
        p = 1
    else:
        p = 0
    return p

In [274]:
m, n = X_norm.shape
#initial_w = 0.01 * (np.random.rand(n) - 0.5)
#initial_b = -8
initial_w = np.zeros(n)
initial_b = 0.
iterations = 1000
alpha = 0.5
lambda_ = 1
w,b, J_history = gradient_descent(X_norm ,y_train, initial_w, initial_b, compute_cost_reg, compute_gradient_reg, alpha, iterations, lambda_)
print(f"b,w found by gradient descent: {b}, {w}")
m,_ = X_norm.shape
for i in range(m):
    print(f"prediction: {predict_single_example(X_norm[i],w,b)}, target value: {y_train[i]}")

Iteration    0: Cost 0.6254733832960967   
Iteration  100: Cost 0.3792436392072075   
Iteration  200: Cost 0.3791773487083124   
Iteration  300: Cost 0.3791749011194815   
Iteration  400: Cost 0.3791747368635576   
Iteration  500: Cost 0.37917472522809503   
Iteration  600: Cost 0.37917472437379696   
Iteration  700: Cost 0.37917472430904   
Iteration  800: Cost 0.37917472430399585   
Iteration  900: Cost 0.37917472430359567   
Iteration  999: Cost 0.37917472430356414   
b,w found by gradient descent: -1.9712583228852207, [ 0.23908358  0.53485781 -0.01714171 -0.0037225   0.25739517  0.04183799
  0.07056384  0.111059    0.07624331  0.04872693  0.31827286 -0.01644995
 -0.06341262 -0.01367449  0.11580257]
prediction: 0, target value: 0
prediction: 0, target value: 0
prediction: 0, target value: 0
prediction: 0, target value: 1
prediction: 0, target value: 0
prediction: 0, target value: 0
prediction: 0, target value: 1
prediction: 0, target value: 0
prediction: 0, target value: 0
predictio

In [275]:
p = predict(X_norm, w, b)
print('Train Accuracy: %f'%(np.mean(p == y_train) * 100))

Train Accuracy: 85.582822
